## ● Python of GIS  MingHsun 2016/03/05
* GIS在Python! GIS的使用者，一般而言都會使用GIS軟體進行操作，但是現在地理資料越來越發多，而資料分析的流程當中，data cleaning是非常重要的一個環節，尤其是地理資料，並不是所有的資料都已經處理成GIS軟體可以讀取格式，尤其是在大量資料需要進行轉換與處理的時候，使用GIS軟體進行操作就會有困難，雖然在ArcGIS之有Model Builder可以使用，但如果可以在程式語言之中使用GIS的操作分析功能+其他的程式語言功能將會非常的方便，而尤其在Python之中不管是資料擷取、分析、探勘、視覺化的模組相當多元，在配合上GIS的功能，可以使用Pytho做出非常多都的目標，以下為Python一些GIS相關的模組。



## Python主要模組操作與介紹:<br>

## pyproj
* 座標投影轉換模組，在地理資料當中空間座標投影是相當重要的一個問題，例如
>1.經緯度座標系統或是投影座標系統的轉換
 2.投影座標系統之間的轉換
* pyproj就是主要解決這些座標系統轉換的問題，也同是可以做些大地測量的計算


## GDAL (Geospatial Data Abstraction Library)
* 事實上在GDAL之中包含了兩個libraries
> 1.GDAL對 Raster Data 進行操作<br>
  2.OGR 對 Vector Data 進行操作


## arcpy
* 可以對於地理資料
>data analysis<br>
data conversion<br>
data management<br>
map automation<br>

* 就是Pyhton化的ArcGIS，透過Pyhton程式語言來使用ArcGIS

## 範例1.Creat Shapefile
### 將大量具有座標的Point資料的csv檔案，使用Python轉換成Shapefile檔案。
> 1.讀取csv資料<br>
  2.進行資料處理<br>
  3.設定所需要之參數<br>
  4.轉換成Shapefile檔案<br>

* 原始抓取在csv之中的Point點資料，並同時做資料的數量確認(在taipeibox資料夾中)

In [3]:
import csv 
total=0
all_uid=[]
y_list=[2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015]
for yy in y_list:
    d=0
    y=str(yy)
    uid_check=[]
    f = open('taipeibox/'+y+'.csv', 'r')  
    for row in csv.reader(f):  
        d=d+1
        total=total+1
        uid_check.append(row[1])
        all_uid.append(row[1])
    f.close()
    print y,len(set(uid_check)),"points",d
print "UNIQUE all userID:",len(set(all_uid)) 
print "SUM of points",total

2001 10 points 97
2002 46 points 741
2003 57 points 1127
2004 89 points 1693
2005 190 points 13286
2006 349 points 19182
2007 611 points 50219
2008 1095 points 90422
2009 2103 points 132092
2010 3108 points 162428
2011 3843 points 201035
2012 3989 points 201645
2013 6004 points 299742
2014 5091 points 257018
2015 3635 points 164934
UNIQUE all userID: 17181
SUM of points 1595661


* 資料格式處理 (座標投影轉換/屬性欄位格式轉換)&輸出.csv(自:taipeibox資料夾 存:taipeibox_process資料夾)

In [4]:
import pyproj
import time
import csv 
# 選擇要抓取的年份
y_list=[2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015]
prj = pyproj.Proj(init="epsg:3828")
point_dict={}#checkdict
point_list=[]#userID_list
for yy in y_list:
    b=0
    a=0
    year=str(yy)
    uid_check=[]
    f = open('taipeibox/'+year+'.csv', 'r')  
    nf = open("taipeibox_process/"+str(year)+".csv","w") 
    rewrite_list=[["timestamp","time_D","time_T","logitude","latitude","X","Y","photo_ID","user_ID"]]
    for point in csv.reader(f):  
        if point[0]!="photoID":
            ph_ID=point[0]
            u_ID=point[1]
            ph_time=point[3]
            x=point[5]
            y=point[4]
            b=b+1
            if x=="0" and y=="0":
                continue
            a=a+1
            split_=ph_time.split()#'2001-10-06 14:58:45'將日期,時間切開
            split_D=split_[0].split('-') #將2014/-2-19 依據'-'切割
            split_T=split_[1].split(':') #將03:00:49 依據':'切割
            split_D[1]=split_D[1].zfill(2)
            split_D[2]=split_D[2].zfill(2)
            D_s=split_D[0]+split_D[1]+split_D[2]
            T_s=split_T[0]+split_T[1]+split_T[2]
            #投影轉換
            photoPx, photoPy = prj(x,y, inverse=False)
            ph_x="{:.7f}".format(photoPx)
            ph_y="{:.7f}".format(photoPy)
            time_tuple=(int(split_D[0]), int(split_D[1]), int(split_D[2]), int(split_T[0]), int(split_T[1]),int(split_T[2]), 0, 0, 0)
            timestamp=time.mktime(time_tuple) 
            temp_list=[timestamp,D_s,T_s,x,y,ph_x,ph_y,ph_ID,u_ID]
            point_list.append(temp_list)
            rewrite_list.append(temp_list)
            if u_ID not in point_dict:
                point_dict.setdefault(u_ID,[temp_list])
            else:
                point_dict[u_ID].append(temp_list)
    w = csv.writer(nf)  
    w.writerows(rewrite_list)
    f.close()
    nf.close()
    print year,"year","before",b,"after",a
#資料缺少 因為當中有x=0 y=0的資料

2001 year before 96 after 96
2002 year before 740 after 740
2003 year before 1126 after 1126
2004 year before 1692 after 1692
2005 year before 13248 after 13248
2006 year before 19156 after 19156
2007 year before 50168 after 50168
2008 year before 90352 after 90347
2009 year before 131990 after 131986
2010 year before 162296 after 162263
2011 year before 200881 after 200623
2012 year before 201500 after 201470
2013 year before 299529 after 299244
2014 year before 256825 after 256629
2015 year before 164819 after 164670


* 本次使對於向量資料進行操綽，使用ogr來將上述的csv資料輸出成Shapefile(自:taipeibox_process 存入:process_shp資料夾)

In [6]:
import os
import osgeo.ogr as ogr
import osgeo.osr as osr
import csv 
y_list=[2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015]
# y_list=[2005]
# Attribute欄位建立與欄位type定義
    layer.CreateField(ogr.FieldDefn("time_D", ogr.OFTInteger))
    layer.CreateField(ogr.FieldDefn("time_T", ogr.OFTInteger))
    layer.CreateField(ogr.FieldDefn("Latitude", ogr.OFTReal))
    layer.CreateField(ogr.FieldDefn("Longitude", ogr.OFTReal))
    layer.CreateField(ogr.FieldDefn("Timestamp", ogr.OFTInteger))
    layer.CreateField(ogr.FieldDefn("X", ogr.OFTReal))
    layer.CreateField(ogr.FieldDefn("Y", ogr.OFTReal))
    # Process the text file and add the attributes and features to the shapefile
    f = open('taipeibox_process/'+y+'.csv', 'r')  
    for point in csv.reader(f):
        if point[0]!="timestamp":
            # create the feature
            feature = ogr.Feature(layer.GetLayerDefn())
            # Set the attributes
            feature.SetField("photoID",point[7])
            feature.SetField("userID", point[8])
            feature.SetField("time_D", point[1])
            feature.SetField("time_T", point[2])
            feature.SetField("Longitude",point[3])
            feature.SetField("Latitude", point[4])
            feature.SetField("Timestamp",point[0])
            feature.SetField("X", point[5])
            feature.SetField("Y", point[6])

            c_logitude=float(point[3])
            c_latitude=float(point[4])
            # create the WKT for the feature using Python string formatting
            wkt = "POINT(%f %f)" %  (float(c_logitude) , float(c_latitude))

            # Create the point from the Well Known Txt
            point = ogr.CreateGeometryFromWkt(wkt)

            # Set the feature geometry using the point
            feature.SetGeometry(point)
            # Create the feature in the layer (shapefile)
            layer.CreateFeature(feature)
            # Destroy the feature to free resources
            feature.Destroy()
    # Destroy the data source to free resources
    data_source.Destroy()

## 範例2. 基礎的GIS操作。
* 將以上步驟，自csv轉換出來的Shapefile資料進行GIS操作
* Clip
* Feature count
* Merge

## Clip  (arcpy)
Syntax
>Clip_analysis (in_features, clip_features, out_feature_class, {cluster_tolerance})

* 讀取taipei shp將資料clip

In [8]:
import arcpy
y_list=[2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015]
for yy in y_list:
    y=str(yy)
    c_shp= "process_shp/"+y+".shp"
    arcpy.Clip_analysis(c_shp, "tpe_shp/taipei_city.shp", "C:/thesis_flickr/clip/"+y+"clip.shp")

## Feature Count 
Syntax
>layer.GetFeatureCount()

* 讀取clip後的資料
* 並對資料進行確認與分析(這些資料是Flickr平台擷取下來，據有Geotag的相片資料，在對於資料進行確認的部分，重點在於照片的位置、拍照時間是否有重疊，對於此類社群媒體平台所截取的資料，資料的清理與確認是非常重要且繁複的步驟，以便後續的資料使用)

In [1]:
import os
from osgeo import ogr
total=0
all_uid=[]
all_t_l=[]
all_l=[]
y_list=[2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015]
for yy in y_list:
    year=str(yy)
    uid_check=[]
    t_l_check=[]
    l_check=[]
    SHP= "clip/"+year+"clip.shp"
    daShapefile = SHP
    driver = ogr.GetDriverByName('ESRI Shapefile')
    dataSource = driver.Open(daShapefile, 0) # 0 means read-only. 1 means writeable. 
    layer = dataSource.GetLayer(0)
    featureCount = layer.GetFeatureCount()
    total=total+featureCount
    for feature in layer:
        userID = feature.GetField('userID')
        timestamp = feature.GetField('timestamp')
        x= feature.GetField('x')
        y= feature.GetField('y')
        total=total+1
        uid_check.append(userID)
        t_l_check.append(str(timestamp)+str(x)+str(y))
        all_uid.append(userID)
        all_t_l.append(userID+str(timestamp)+str(x)+str(y))
        l_check.append(str(x)+str(y))
        all_l.append(str(x)+str(y))
    # Destroy the data source to free resources
    dataSource.Destroy()
    print year,"points",featureCount,"unique uid",len(set(uid_check))
    print "unique time_location",len(set(t_l_check))
    print "unique_location",len(set(l_check))
print "SUM of points",total
print "UNIQUE all userID:",len(set(all_uid))
print "UNIQUE all time_location:",len(set(all_t_l))
print "UNIQUE all location:",len(set(all_l))

2001 points 94 unique uid 8
unique time_location 92
unique_location 19
2002 points 699 unique uid 40
unique time_location 673
unique_location 113
2003 points 1051 unique uid 51
unique time_location 1018
unique_location 88
2004 points 1547 unique uid 78
unique time_location 1510
unique_location 149
2005 points 12866 unique uid 172
unique time_location 12775
unique_location 875
2006 points 16777 unique uid 317
unique time_location 16508
unique_location 2295
2007 points 44124 unique uid 563
unique time_location 43448
unique_location 7438
2008 points 83471 unique uid 986
unique time_location 80738
unique_location 15322
2009 points 116059 unique uid 1886
unique time_location 114014
unique_location 23326
2010 points 144293 unique uid 2770
unique time_location 141822
unique_location 32205
2011 points 176325 unique uid 3487
unique time_location 172328
unique_location 47292
2012 points 169001 unique uid 3635
unique time_location 160694
unique_location 54001
2013 points 239293 unique uid 5343
un

## Merge  (arcpy)
Syntax
>Merge_management (inputs, output, {field_mappings})<br>

* 將多張Shapefile資料進行Merge

In [1]:
import arcpy
arcpy.env.workspace = "C:/thesis_flickr/clip"
merge_list=[]
y_list=[2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015]
for yy in y_list:
    year=str(yy)
    shp= year+"clip.shp"
    merge_list.append(shp)
print merge_list
# #delete shp
if os.path.exists("merge.shp"):
    driver.DeleteDataSource("merge.shp")
arcpy.Merge_management(merge_list, "C:/thesis_flickr/merge.shp")

* Get merged feature count

In [4]:
import os
from osgeo import ogr
SHP= "merge.shp"
daShapefile = SHP
driver = ogr.GetDriverByName('ESRI Shapefile')
dataSource = driver.Open(daShapefile, 0) # 0 means read-only. 1 means writeable. 
layer = dataSource.GetLayer(0)
featureCount = layer.GetFeatureCount()
print featureCount
# Destroy the data source to free resources
dataSource.Destroy()

1352500
